In [1]:
import optuna
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OrdinalEncoder
from xgboost import XGBClassifier

In [2]:
# Read dataset
df = pd.read_csv("predictive_maintenance.csv").drop(['Target'], axis=1).dropna().drop_duplicates()

In [3]:
# Apply LabelEncoder to target column
df['Failure Type'] = LabelEncoder().fit_transform(df['Failure Type'])

# Apply OrdinalEncoder to categorical features
categorical_features = df.select_dtypes(include='object').columns
df[categorical_features] = OrdinalEncoder().fit_transform(df[categorical_features])

# Apply MinMaxScaler to all continous features
continuous_features = df.select_dtypes(include=['float64', 'int64']).columns
df[continuous_features] = MinMaxScaler().fit_transform(df[continuous_features])

In [4]:
# Remove special characters from column names
df.columns = df.columns.str.replace('[^\w\s]', '', regex=True)

In [5]:
# Split dataframe to train and test subsets
X = df.drop(['Failure Type'], axis=1)
y = df['Failure Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
# Perform hyperparameter optimization
def objective(trial):
    try:
        eta = trial.suggest_float('eta', 0.01, 0.3, step=0.01)
        max_depth = trial.suggest_int('max_depth', 3, 10)
        subsample = trial.suggest_float('subsample', 0.5, 1.0, step=0.1)
        colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0, step=0.1)
        min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
        gamma = trial.suggest_float('gamma', 0.0, 1.0, step=0.1)
        reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0, step=0.1)
        reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0, step=0.1)
        clf = XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            eta=eta,
            max_depth=max_depth,
            subsample=subsample,
            colsample_bytree=colsample_bytree,
            min_child_weight=min_child_weight,
            gamma=gamma,
            reg_lambda=reg_lambda,
            reg_alpha=reg_alpha,
            n_jobs=-1,
            random_state=42
        )
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)
        trial.set_user_attr('classification_report', report)
        return accuracy
    except:
        raise optuna.TrialPruned()

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000, n_jobs=-1)

[I 2023-06-02 13:49:23,031] A new study created in memory with name: no-name-1574c317-b3d7-4cf8-aa8c-3d28f02fa1d7
[I 2023-06-02 13:49:24,653] Trial 12 finished with value: 0.9833333333333333 and parameters: {'eta': 0.27, 'max_depth': 6, 'subsample': 0.5, 'colsample_bytree': 1.0, 'min_child_weight': 8, 'gamma': 0.1, 'reg_lambda': 0.6000000000000001, 'reg_alpha': 0.8}. Best is trial 12 with value: 0.9833333333333333.
[I 2023-06-02 13:49:24,915] Trial 13 finished with value: 0.9833333333333333 and parameters: {'eta': 0.25, 'max_depth': 5, 'subsample': 0.9, 'colsample_bytree': 0.7, 'min_child_weight': 10, 'gamma': 0.8, 'reg_lambda': 0.7000000000000001, 'reg_alpha': 0.1}. Best is trial 12 with value: 0.9833333333333333.
[I 2023-06-02 13:49:24,934] Trial 14 finished with value: 0.9833333333333333 and parameters: {'eta': 0.22, 'max_depth': 9, 'subsample': 0.7, 'colsample_bytree': 0.7, 'min_child_weight': 9, 'gamma': 0.4, 'reg_lambda': 0.4, 'reg_alpha': 0.9}. Best is trial 12 with value: 0.983

[I 2023-06-02 13:49:28,195] Trial 28 finished with value: 0.985 and parameters: {'eta': 0.17, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.5, 'min_child_weight': 1, 'gamma': 0.6000000000000001, 'reg_lambda': 0.0, 'reg_alpha': 0.0}. Best is trial 28 with value: 0.985.
[I 2023-06-02 13:49:28,486] Trial 29 finished with value: 0.984 and parameters: {'eta': 0.3, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.5, 'min_child_weight': 1, 'gamma': 0.6000000000000001, 'reg_lambda': 0.0, 'reg_alpha': 0.0}. Best is trial 28 with value: 0.985.
[I 2023-06-02 13:49:28,555] Trial 30 finished with value: 0.983 and parameters: {'eta': 0.17, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 6, 'gamma': 0.6000000000000001, 'reg_lambda': 0.0, 'reg_alpha': 0.0}. Best is trial 28 with value: 0.985.
[I 2023-06-02 13:49:28,750] Trial 20 finished with value: 0.9826666666666667 and parameters: {'eta': 0.08, 'max_depth': 7, 'subsample': 0.7, 'colsample_bytree': 0.6

[I 2023-06-02 13:49:33,450] Trial 57 finished with value: 0.9833333333333333 and parameters: {'eta': 0.14, 'max_depth': 4, 'subsample': 0.9, 'colsample_bytree': 1.0, 'min_child_weight': 4, 'gamma': 1.0, 'reg_lambda': 0.1, 'reg_alpha': 0.2}. Best is trial 28 with value: 0.985.
[I 2023-06-02 13:49:33,575] Trial 52 finished with value: 0.984 and parameters: {'eta': 0.13, 'max_depth': 5, 'subsample': 0.9, 'colsample_bytree': 1.0, 'min_child_weight': 4, 'gamma': 0.2, 'reg_lambda': 0.2, 'reg_alpha': 0.1}. Best is trial 28 with value: 0.985.
[I 2023-06-02 13:49:33,615] Trial 53 finished with value: 0.984 and parameters: {'eta': 0.13, 'max_depth': 5, 'subsample': 0.9, 'colsample_bytree': 1.0, 'min_child_weight': 4, 'gamma': 0.30000000000000004, 'reg_lambda': 0.1, 'reg_alpha': 0.2}. Best is trial 28 with value: 0.985.
[I 2023-06-02 13:49:33,679] Trial 54 finished with value: 0.984 and parameters: {'eta': 0.14, 'max_depth': 5, 'subsample': 0.9, 'colsample_bytree': 1.0, 'min_child_weight': 4, 'ga

[I 2023-06-02 13:49:38,802] Trial 85 finished with value: 0.9823333333333333 and parameters: {'eta': 0.06999999999999999, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 1, 'gamma': 0.8, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 28 with value: 0.985.
[I 2023-06-02 13:49:39,125] Trial 81 finished with value: 0.9853333333333333 and parameters: {'eta': 0.16, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 1, 'gamma': 0.8, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 81 with value: 0.9853333333333333.
[I 2023-06-02 13:49:39,542] Trial 82 finished with value: 0.9853333333333333 and parameters: {'eta': 0.05, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 0.9, 'min_child_weight': 2, 'gamma': 0.8, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.30000000000000004}. Best is trial 81 with value: 0.9853333333333333.
[I 2023-06-02 13:49:39,759] Trial 83 finished with value: 0.9843

[I 2023-06-02 13:49:44,634] Trial 113 finished with value: 0.9853333333333333 and parameters: {'eta': 0.19, 'max_depth': 3, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 5, 'gamma': 0.5, 'reg_lambda': 0.0, 'reg_alpha': 0.1}. Best is trial 89 with value: 0.9856666666666667.
[I 2023-06-02 13:49:45,134] Trial 110 finished with value: 0.9843333333333333 and parameters: {'eta': 0.19, 'max_depth': 9, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 5, 'gamma': 0.5, 'reg_lambda': 0.4, 'reg_alpha': 0.1}. Best is trial 89 with value: 0.9856666666666667.
[I 2023-06-02 13:49:45,327] Trial 111 finished with value: 0.9843333333333333 and parameters: {'eta': 0.2, 'max_depth': 8, 'subsample': 1.0, 'colsample_bytree': 1.0, 'min_child_weight': 5, 'gamma': 0.5, 'reg_lambda': 0.4, 'reg_alpha': 0.1}. Best is trial 89 with value: 0.9856666666666667.
[I 2023-06-02 13:49:45,395] Trial 112 finished with value: 0.984 and parameters: {'eta': 0.27, 'max_depth': 9, 'subsample': 1.0,

...

[I 2023-06-02 13:51:57,538] Trial 865 finished with value: 0.987 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:51:57,837] Trial 866 finished with value: 0.987 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:51:57,910] Trial 867 finished with value: 0.987 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:51:58,288] Trial 868 finished with value: 0.9813333333333333 and parameters: {'eta': 0.01, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytr

[I 2023-06-02 13:52:03,367] Trial 891 finished with value: 0.9853333333333333 and parameters: {'eta': 0.28, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:03,623] Trial 892 finished with value: 0.9853333333333333 and parameters: {'eta': 0.28, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:03,806] Trial 893 finished with value: 0.9853333333333333 and parameters: {'eta': 0.28, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:04,171] Trial 894 finished with value: 0.9856666666666667 and parameters: {'eta': 0.29000000000

[I 2023-06-02 13:52:09,236] Trial 919 finished with value: 0.986 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.4, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:09,504] Trial 920 finished with value: 0.986 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.4, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:09,547] Trial 921 finished with value: 0.986 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.4, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:09,817] Trial 922 finished with value: 0.986 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0

[I 2023-06-02 13:52:14,892] Trial 948 finished with value: 0.986 and parameters: {'eta': 0.29000000000000004, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:15,099] Trial 946 finished with value: 0.9843333333333333 and parameters: {'eta': 0.29000000000000004, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.9, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:15,520] Trial 950 finished with value: 0.9853333333333333 and parameters: {'eta': 0.29000000000000004, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.0, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.6000000000000001}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:15,654] Trial 949 finished with value: 0.986 and

[I 2023-06-02 13:52:20,553] Trial 973 finished with value: 0.9856666666666667 and parameters: {'eta': 0.3, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:20,870] Trial 974 finished with value: 0.9856666666666667 and parameters: {'eta': 0.3, 'max_depth': 6, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:20,940] Trial 975 finished with value: 0.987 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0}. Best is trial 674 with value: 0.987.
[I 2023-06-02 13:52:21,446] Trial 976 finished with value: 0.987 and parameters: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'co

In [7]:
# Print best trial
print(f"\nBest trial: {study.best_trial.params} => {study.best_trial.values[0]:.3f}")
print(f"\n{pd.DataFrame.from_dict(study.best_trial.user_attrs['classification_report']).round(3).T}")


Best trial: {'eta': 0.3, 'max_depth': 5, 'subsample': 1.0, 'colsample_bytree': 0.8, 'min_child_weight': 1, 'gamma': 0.1, 'reg_lambda': 0.30000000000000004, 'reg_alpha': 0.0} => 0.987

              precision  recall  f1-score   support
0                 0.815   0.957     0.880    23.000
1                 0.990   0.998     0.994  2903.000
2                 0.882   0.682     0.769    22.000
3                 0.929   0.839     0.881    31.000
4                 0.000   0.000     0.000     7.000
5                 0.500   0.071     0.125    14.000
accuracy          0.987   0.987     0.987     0.987
macro avg         0.686   0.591     0.608  3000.000
weighted avg      0.983   0.987     0.984  3000.000
